<a href="https://colab.research.google.com/github/dipnarayan501/Deep_learning/blob/main/CS6910_Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.datasets import mnist, fashion_mnist
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

Question 1 : Prepare dataset from fashion mnist 

In [ ]:
#Question1 Code here

In [ ]:
def prepare_dataset():
    (X, y), (X_test, y_test) = fashion_mnist.load_data()

    # Reshaping the data matrices
    X = X.reshape(X.shape[0], -1)
    X_test = X_test.reshape(X_test.shape[0], -1)

    # Normalizing the pixel intensities
    X = X/255.0
    X_test = X_test/255.0
    
    
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)
    
    y_train_encode = np.zeros((10,y_train.shape[0]))
    y_train_encode[y_train, np.array(list(range(y_train.shape[0])))] = 1

    y_val_encode = np.zeros((10,y_val.shape[0]))
    y_val_encode[y_val, np.array(list(range(y_val.shape[0])))] = 1

    
    
    print("Number of examples in training set")
    print(X_train.shape)
    print(y_train_encode.shape)
    print(y_train.shape)
    
    print("No of examples in validation set")
    print(X_val.shape)
    print(y_val.shape)
    print("No of examples in test set")
    print(X_test.shape)
    print(y_test.shape)
    
    return X_train.T, y_train_encode, y_train.T ,X_val.T, y_val.T, X_test.T, y_test.T, y_val_encode


X_train, y_train_encode, y_train ,X_val, y_val, X_test, y_test,y_val_encode = prepare_dataset()

4431872/4422102 [==============================] - 0s 0us/step
Number of examples in training set
(54000, 784)
(10, 54000)
(54000,)
No of examples in validation set
(6000, 784)
(6000,)
No of examples in test set
(10000, 784)
(10000,)


In [ ]:
def sigmoid(x):
    return 1. / (1.+np.exp(-x))
def relu(x):
    return np.maximum(0,x)
def tanh(x):
    return 1.*(np.exp(x)+np.exp(-x))/(np.exp(x)-np.exp(-x))
def softmax(x):
    soft = np.zeros(x.shape)
    for i in range(0, x.shape[1]):
        numr = np.exp(x[:, i])
        soft[:, i] = numr/np.sum(numr)
    return soft


#Derivatives of sigmoid function
def sigmoid_derivative(x):
    return sigmoid(x) * (1-sigmoid(x))
def relu_derivative(x):
    return 1*(x>0)
def tanh_derivative(x):
    return (1 - (np.tanh(x)**2))





#Initialisation of parameters
def initialize_parameters(input_layer, hidden_layer, output_layer,initilisation):
    weights = []
    biases  = []
    layers = [input_layer] + hidden_layer + [output_layer]
    for i in range(len(hidden_layer)+1):
        if initilisation == 'random':
            weights.append(np.random.rand(layers[i+1], layers[i])*0.01)
        if initilisation == 'xavier':
            weights.append(np.random.randn(layers[i+1],layers[i])*np.sqrt(2/layers[i+1]))
        biases.append(np.random.randn(layers[i+1],1))
    return weights, biases


In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error

class ANN:

    def __init__(self,n_inputs,n_hidden,n_outputs,initialisation, activation,loss):
           
        self.input_layer   = n_inputs
        self.output_layer  = n_outputs
        self.hidden_layer   = n_hidden
        self.activation = activation
        self.loss_function = loss
        self.weights    = []
        self.biases     = []
        self.initialisation = initialisation
        self.weights,self.biases = initialize_parameters(self.input_layer,self.hidden_layer,self.output_layer,self.initialisation)

    
    
    #Forward Propogation code here
    def forward_propagation(X, weights, biases,activation):

        A = []  #pre-activation
       
        H  = []    # post activation
        W      = weights
        b      = biases

        k=0
        A.append(np.dot(W[k],X)+b[k])
        if activation == 'sigmoid':
          H.append(sigmoid(A[k]))
        elif activation == 'tanh':
          H.append(tanh(A[k]))
        elif activation == 'relu':
          H.append(relu(A[k]))


        for k in range(1,len(hidden_layer)):
            A.append(np.dot(W[k],H[k-1])+b[k])
            if activation == 'sigmoid':
              H.append(sigmoid(A[k]))
            elif activation == 'tanh':
              H.append(tanh(A[k]))
            elif activation == 'relu':
              H.append(relu(A[k]))
            elif activation == 'leaky_relu':
              H.append(leaky_relu(A[k]))

        k=len(hidden_layer)
        A.append(np.dot(W[k],H[k-1])+b[k])
        H.append(softmax(A[k]))

        return A , H
        A, H = forward_propagation(X_train, weights, biases, 'sigmoid')
        for i in range(len(A)):
          print(A[i].shape)
          for i in range(len(H)):
            print(H[i].shape)
    


     #Back propogation
    def back_propagation(self,X_train,y_train):

        g_a  = [0]*(len(self.hidden_layer)+1)
        g_h  = [0]*(len(self.hidden_layer)+1)
        g_w  = [0]*(len(self.weights))
        g_b  = [0]*(len(self.biases))

        n_samples = X_train.shape[1] 

        for k in reversed(range(len(self.hidden_layer)+1)):
            if k == len(self.hidden_layer):
              if self.loss_function == 'cross_entropy':
                  g_a[k] = self.H[k]  - y_train  
              elif self.loss_function == 'square_loss': 
                  g_a[k] = (self.H[k] - y_train) * self.H[k] * (1 - self.H[k]) 
                
            else:
                g_h[k] = (1/n_samples)*np.matmul(self.weights[k+1].T,g_a[k+1])
                if self.activation == 'sigmoid':
                  g_a[k] = (1/n_samples)*np.multiply(g_h[k],sigmoid_derivative(self.A[k]))
                elif self.activation == 'tanh':
                  g_a[k] = (1/n_samples)*np.multiply(g_h[k],tanh_derivative(self.A[k]))
                elif self.activation == 'relu':
                  g_a[k] = (1/n_samples)*np.multiply(g_h[k],relu_derivative(self.A[k]))
                

            if k == 0:
                g_w[k] = (1/n_samples)*np.matmul(g_a[k],X_train.T) 
            else:
                g_w[k] = (1/n_samples)*np.matmul(g_a[k],self.H[k-1].T)

            g_b[k]  = (1/n_samples)*np.sum(g_a[k], axis=1, keepdims = True)
        return g_w,g_b
    def predict(self, X,y ):
      output =  self.forward_propagation(X)
      out_class=(np.argmax(output,axis=0))
      accuracy = round(accuracy_score(y, out_class) * 100, 3)
      return accuracy
    def fit_predict(self,X_train,y_train,X_val ,y_val ,epochs,learning_rate,optimiser='gd',batch_size = 64,lambd=0.0005): 
      steps = 0
      pre_update_w = np.multiply(self.weights,0)
      pre_update_b = np.multiply(self.biases,0)
      update_w = np.multiply(self.weights,0)
      update_b = np.multiply(self.biases,0)
      vw = 0.0
      vb = 0.0
      eps = 1e-8
      a1 =0.0
      gamma = 0.9
      beta = 0.999
      beta1 = 0.9
      beta2 = 0.999
      m_t, v_t, m_hat_w, v_hat_w, m_b,v_b,m_hat_b,v_hat_b = 0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0 

      for i in range(epochs):
        

            for batch in range(0, X_train.shape[1], batch_size):

              train_b_imag =  X_train[:,batch:batch+batch_size]
              train_l_imag =  y_train_encode[:,batch:batch+batch_size]
              output =  self.forward_propagation(train_b_imag)
              g_weights,g_biases = self.back_propagation(train_b_imag,train_l_imag)

              #gradient descent
              if optimiser == 'gd':
                 self.weights = self.weights - np.multiply(learning_rate,g_weights) - np.multiply(learning_rate*lambd,self.weights)
                 self.biases = self.biases - np.multiply(learning_rate,g_biases)
              #Momemtum gradiend descent code



              #NAG code here



              #RMS prop code here 
              if optimiser == 'rmsprop': 

                 vw = np.multiply(vw,beta) + np.multiply(1-beta,np.power(g_weights,2))
                 vb = np.multiply(vb,beta) + np.multiply(1-beta,np.power(g_biases,2))

          
                 update_w = np.multiply(g_weights,learning_rate/np.power(vw+eps,1/2))
                 update_b = np.multiply(g_biases,learning_rate/np.power(vb+eps,1/2))
                 
                 self.weights = self.weights - update_w - np.multiply(learning_rate*lambd,self.weights)
                 self.biases = self.biases - update_b
              #Adam code here    
              if optimiser == 'adam':
                 m_t = np.multiply(beta1,m_t) + np.multiply(1-beta1,g_weights)
                 v_t = np.multiply(beta2,v_t) + np.multiply(1-beta2,np.power(g_weights,2))
                 m_b = np.multiply(beta1,m_b) + np.multiply(1-beta1,g_biases)
                 v_b = np.multiply(beta2,v_b) + np.multiply(1-beta2,np.power(g_biases,2))
                
                 m_hat_w = m_t/(1 - np.power(beta1,i+1))
                 m_hat_b = m_b/(1 - np.power(beta1,i+1))
                
                 v_hat_w = v_t/(1 - np.power(beta2,i+1))
                 v_hat_b = v_b/(1 - np.power(beta2,i+1))
                 update_w = (learning_rate / np.power(v_hat_w + eps, 1/2)) * m_hat_w
                 update_b = (learning_rate / np.power(v_hat_b + eps, 1/2)) * m_hat_b
                 self.weights = self.weights - update_w - np.multiply(learning_rate*lambd,self.weights)
                 self.biases = self.biases - update_b
              #Nadam code here
              #start here

                
            
            #Training loss for full dataset
            predicted_train = self.forward_propagation(X_train)
            predicted_train_label=(np.argmax(predicted_train,axis=0)) 
             
            acc1 = round(accuracy_score(y_train, predicted_train_label) * 100, 3)

            predicted_val = self.forward_propagation(X_val)
            predicted_val_label=(np.argmax(predicted_val,axis=0))
              
            acc2 = round(accuracy_score(y_val, predicted_val_label) * 100, 3)

            a =self.weights[1:len(self.hidden_layer)]
            b = np.sum([(np.sum((a[i]**2).reshape(1,-1))) for i in range(len(a))]) 
            if self.loss_function == 'cross_entropy':
              train_loss = (-np.sum(np.multiply(y_train_encode,np.log(predicted_train)))+((lambd/2.)*b))/y_train.shape[0]
              val_loss = (-np.sum(np.multiply(y_val_encode,np.log(predicted_val)))+((lambd/2.)*b))/y_val.shape[0]
            print('Epoch {}: training_accuracy = {:.2f} %, training_loss= {:.5f}  ,Validation accuracy = {:.2f} ,Validation loss = {:.5f}'.format(i,acc1,train_loss,acc2,val_loss))
          
            wandb.log({"val_accuracy": acc2,"accuracy": acc1,"steps":epochs,"train_loss":train_loss,"val_loss":val_loss},)

            

         
      return acc1,acc2,train_loss,val_loss
    
